#Parquet Version of BacktestEngine 


claude version with multiple corrections and fixes


In [19]:
# Wolf's AlgoHaus Backtester v6.0 - Professional Edition
# Created for: Wolf Guzman
# Features: Real Data Validation, Risk Management, QuantStats Analysis, Retro DOS Metrics
# Professional Forex Backtesting with Parquet Data

import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog, scrolledtext
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import webbrowser
import os
import tempfile
import inspect
import pathlib
import threading
import queue
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import logging

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("green")
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


In [20]:

# ======================================================================
# 1. FOREX CALCULATOR
# ======================================================================
class ForexCalculator:
    """Handle all forex calculations - Wolf Guzman's Trading System"""
   
    LEVERAGE_OPTIONS = [1, 10, 20, 30, 50, 100, 200, 500]
   
    PIP_VALUES = {
        # Major Pairs
        'EUR/USD': 0.0001, 'GBP/USD': 0.0001, 'USD/JPY': 0.01,
        'USD/CHF': 0.0001, 'USD/CAD': 0.0001, 'AUD/USD': 0.0001,
        'NZD/USD': 0.0001,
        # Cross Pairs - JPY
        'EUR/JPY': 0.01, 'GBP/JPY': 0.01, 'AUD/JPY': 0.01,
        'NZD/JPY': 0.01, 'CHF/JPY': 0.01, 'CAD/JPY': 0.01,
        # Cross Pairs - EUR
        'EUR/GBP': 0.0001, 'EUR/CHF': 0.0001, 'EUR/AUD': 0.0001,
        # Cross Pairs - GBP
        'GBP/CHF': 0.0001,
        # Cross Pairs - AUD
        'AUD/CHF': 0.0001, 'AUD/NZD': 0.0001,
        # Cross Pairs - NZD
        'NZD/CHF': 0.0001,
        # Cross Pairs - CAD
        'CAD/CHF': 0.0001,
        # Exotic
        'USD/THB': 0.01
    }
    
    # Hardcoded date ranges from Wolf's data for faster loading
    DATA_RANGES = {
        'AUD/CHF': ('2014-12-09', '2025-10-06'),
        'AUD/JPY': ('2014-12-10', '2025-10-06'),
        'AUD/NZD': ('2014-12-10', '2025-10-06'),
        'AUD/USD': ('2014-12-09', '2025-10-06'),
        'CAD/CHF': ('2014-12-10', '2025-10-06'),
        'CHF/JPY': ('2014-12-08', '2025-10-06'),
        'EUR/AUD': ('2014-12-09', '2025-10-06'),
        'EUR/CHF': ('2014-12-10', '2025-10-06'),
        'EUR/GBP': ('2014-12-08', '2025-10-06'),
        'EUR/JPY': ('2014-12-10', '2025-10-06'),
        'EUR/USD': ('2014-12-09', '2025-10-06'),
        'GBP/CHF': ('2014-12-10', '2025-10-06'),
        'GBP/USD': ('2014-12-10', '2025-10-06'),
        'NZD/CHF': ('2014-12-09', '2025-10-06'),
        'NZD/JPY': ('2014-12-08', '2025-10-06'),
        'NZD/USD': ('2014-12-10', '2025-10-06'),
        'USD/CAD': ('2014-12-10', '2025-10-06'),
        'USD/CHF': ('2014-12-09', '2025-10-06'),
        'USD/JPY': ('2014-12-09', '2025-10-06'),
        'USD/THB': ('2014-12-03', '2025-10-06'),
    }
    
    USD_MAJORS = {
        'EUR/USD', 'GBP/USD', 'AUD/USD', 'NZD/USD', 
        'USD/JPY', 'USD/CHF', 'USD/CAD', 'USD/THB'
    }
    
    @staticmethod
    def is_usd_major(pair):
        return pair in ForexCalculator.USD_MAJORS
   
    @staticmethod
    def calculate_pip_value_in_usd(pair, unit_size, current_price, conversion_rate=1.0):
        """Calculate pip value in USD for given position size"""
        pip_size = ForexCalculator.PIP_VALUES.get(pair, 0.0001) 
        
        if pair.endswith('/USD'):
            # Direct quote: EUR/USD, GBP/USD, etc.
            pip_value = pip_size * unit_size
        elif pair.startswith('USD/'):
            # Indirect quote: USD/JPY, USD/CHF, etc.
            pip_value = (pip_size / current_price) * unit_size
        else:
            # Cross pair: EUR/GBP, GBP/JPY, etc.
            pip_value_in_base = pip_size * unit_size
            pip_value = pip_value_in_base * conversion_rate 
        
        return pip_value
   
    @staticmethod
    def calculate_margin_required(pair, unit_size, current_price, leverage, conversion_rate=1.0):
        """Calculate margin required for position"""
        if pair.endswith('/USD'):
            position_value = unit_size * current_price
        elif pair.startswith('USD/'):
            position_value = unit_size
        else:
            position_value = unit_size * conversion_rate 
        
        margin_required = position_value / leverage
        return margin_required
    
    @staticmethod
    def calculate_position_size(balance, risk_pct, sl_pips, pair, price, conversion_rate=1.0):
        """Calculate position size based on risk percentage"""
        risk_amount = balance * (risk_pct / 100)
        pip_val_per_unit = ForexCalculator.calculate_pip_value_in_usd(pair, 1, price, conversion_rate)
        if pip_val_per_unit <= 0:
            return 0
        size = risk_amount / (sl_pips * pip_val_per_unit)
        return max(1000, int(round(size / 1000)) * 1000)


In [21]:

# ======================================================================
# 2. DATA LOADING WITH VALIDATION
# ======================================================================
def detect_available_pairs(base_folder: pathlib.Path):
    """Scan for available forex pair folders and return valid pairs"""
    pairs = set()
    
    # Check if base folder exists
    if not base_folder.exists():
        logging.error(f"Base folder does not exist: {base_folder}")
        return []
    
    logging.info(f"Scanning for pairs in: {base_folder}")
    
    # Scan for subfolder names like EUR_USD, GBP_USD, etc.
    for subfolder in base_folder.iterdir():
        if subfolder.is_dir() and subfolder.name not in ['README.TXT', '__pycache__']:
            folder_name = subfolder.name
            # Check if folder name matches pair pattern (XXX_XXX)
            if '_' in folder_name and len(folder_name.split('_')) == 2:
                parts = folder_name.split('_')
                # Verify both parts are 3 letters (currency codes)
                if len(parts[0]) == 3 and len(parts[1]) == 3:
                    # Verify folder contains parquet files
                    parquet_files = list(subfolder.glob("*.parquet"))
                    if parquet_files:
                        pair = folder_name.replace('_', '/')
                        pairs.add(pair)
                        logging.info(f"Found pair: {pair} with {len(parquet_files)} parquet file(s)")
                    else:
                        logging.warning(f"Folder {folder_name} has no parquet files")
    
    if not pairs:
        logging.warning("No valid pairs found!")
    else:
        logging.info(f"Total pairs found: {len(pairs)}")
    
    return sorted(list(pairs))

def get_data_date_range(pair_name: str, base_folder: pathlib.Path):
    """Get actual date range from parquet file in pair's subfolder"""
    
    # First, try hardcoded values for instant response
    if pair_name in ForexCalculator.DATA_RANGES:
        start_str, end_str = ForexCalculator.DATA_RANGES[pair_name]
        from datetime import datetime
        start = datetime.strptime(start_str, '%Y-%m-%d').date()
        end = datetime.strptime(end_str, '%Y-%m-%d').date()
        logging.info(f"{pair_name}: Using cached date range {start} to {end}")
        return start, end
    
    # Fallback: read from file (slower but accurate if data updated)
    try:
        # Convert pair name to folder name (EUR/USD -> EUR_USD)
        pair_folder_name = pair_name.replace('/', '_')
        pair_folder = base_folder / pair_folder_name
        
        # Check if pair folder exists
        if not pair_folder.exists() or not pair_folder.is_dir():
            logging.warning(f"Folder not found: {pair_folder}")
            return None, None
        
        # Get parquet files from pair's folder
        parquet_files = list(pair_folder.glob("*.parquet"))
        
        if not parquet_files:
            logging.warning(f"No parquet files in {pair_folder}")
            return None, None
       
        df = pd.read_parquet(parquet_files[0], engine='pyarrow')
        
        # Find datetime column
        datetime_col = None
        for col in df.columns:
            if 'datetime' in col.lower() or 'date' in col.lower() or 'time' in col.lower():
                datetime_col = col
                break
        
        if datetime_col:
            df[datetime_col] = pd.to_datetime(df[datetime_col], errors='coerce', utc=True)
            df = df.dropna(subset=[datetime_col])
            df[datetime_col] = df[datetime_col].dt.tz_localize(None)
            start = df[datetime_col].min().date()
            end = df[datetime_col].max().date()
            logging.info(f"{pair_name}: Data from {start} to {end} (read from file)")
            return start, end
        
        return None, None
    except Exception as e:
        logging.error(f"Error getting date range for {pair_name}: {e}")
        return None, None

def load_pair_data(pair_name: str, base_folder: pathlib.Path, start_date: datetime, end_date: datetime, timeframe: str):
    """Load and validate parquet data from pair-specific subfolder"""
    
    # Convert pair name to folder name (EUR/USD -> EUR_USD)
    pair_folder_name = pair_name.replace('/', '_')
    
    # Construct the full path to the pair's folder
    pair_folder = base_folder / pair_folder_name
    
    logging.info(f"=" * 50)
    logging.info(f"Loading pair: {pair_name}")
    logging.info(f"Base folder: {base_folder}")
    logging.info(f"Pair folder name: {pair_folder_name}")
    logging.info(f"Full pair folder path: {pair_folder}")
    logging.info(f"Pair folder exists: {pair_folder.exists()}")
    logging.info(f"Is directory: {pair_folder.is_dir() if pair_folder.exists() else 'N/A'}")
    
    # Check if pair folder exists
    if not pair_folder.exists():
        raise FileNotFoundError(f"Folder does not exist: {pair_folder}")
    
    if not pair_folder.is_dir():
        raise FileNotFoundError(f"Path is not a directory: {pair_folder}")
    
    # Get all parquet files in the pair's folder
    parquet_files = list(pair_folder.glob("*.parquet"))
    logging.info(f"Parquet files found: {len(parquet_files)}")
    
    if not parquet_files:
        # List what IS in the folder
        folder_contents = list(pair_folder.iterdir())
        logging.error(f"Folder contents: {[f.name for f in folder_contents]}")
        raise FileNotFoundError(f"No PARQUET files found in {pair_folder}")
    
    # Use the first parquet file
    parquet_path = parquet_files[0]
    logging.info(f"Loading file: {parquet_path.name}")
    
    df = pd.read_parquet(parquet_path, engine='pyarrow')
    
    if df.empty:
        raise ValueError(f"PARQUET file is empty: {parquet_path}")
   
    # Map columns
    cols_lower = [c.strip().lower() for c in df.columns]
    col_map = {
        'datetime': ['datetime', 'date', 'time', 'timestamp', 'date_time', 'index'],
        'open': ['open', 'o'],
        'high': ['high', 'h'],
        'low': ['low', 'l'],
        'close': ['close', 'c', 'last'],
        'volume': ['volume', 'vol', 'v']
    }
   
    rename = {}
    for target, aliases in col_map.items():
        for alias in aliases:
            if any(alias in col for col in cols_lower):
                orig = next(col for col in df.columns if alias in col.lower())
                rename[orig] = target
                break
        else:
            if target != 'volume':
                raise KeyError(f"Column for '{target}' not found in {parquet_path}")
   
    df = df.rename(columns=rename)
    
    if 'volume' not in df.columns:
        df['volume'] = 1000
   
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', utc=True)
    df = df.dropna(subset=['datetime'])
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    
    df = df.drop_duplicates(subset='datetime', keep='first')
    df = df.sort_values('datetime')
   
    # Get actual data range
    actual_start = df['datetime'].min().date()
    actual_end = df['datetime'].max().date()
    
    logging.info(f"Data range: {actual_start} to {actual_end}")
    
    # Validate user date range
    if start_date.date() < actual_start:
        logging.warning(f"Start date {start_date.date()} before data start {actual_start}, adjusting to {actual_start}")
        start_date = datetime.combine(actual_start, datetime.min.time())
    
    if end_date.date() > actual_end:
        logging.warning(f"End date {end_date.date()} after data end {actual_end}, adjusting to {actual_end}")
        end_date = datetime.combine(actual_end, datetime.min.time())
   
    df = df.set_index('datetime')
    user_start = max(pd.Timestamp(start_date.date()), df.index.min())
    user_end = min(pd.Timestamp(end_date.date()) + pd.Timedelta(hours=23, minutes=59, seconds=59), df.index.max())
    df = df.loc[user_start:user_end].copy()
   
    if df.empty:
        raise ValueError(f"No data in range {start_date.date()} to {end_date.date()}")
   
    # Resample if needed
    if timeframe != '1min':
        rule = {'5min': '5T', '15min': '15T', '1hr': '1H', '1Day': '1D'}.get(timeframe, '1T')
        logging.info(f"Resampling to {timeframe}")
        df = df.resample(rule).agg({
            'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'
        }).dropna()
   
    df = df.reset_index()
    df['date'] = df['datetime'].dt.date
   
    # Calculate previous day levels
    daily = df.groupby('date').agg({
        'high': 'max', 'low': 'min', 'close': 'last'
    })
    daily.columns = ['day_high', 'day_low', 'day_close']
    daily['prev_high'] = daily['day_high'].shift(1)
    daily['prev_low'] = daily['day_low'].shift(1)
    daily['prev_close'] = daily['day_close'].shift(1)
   
    daily = daily.reset_index()
    df = pd.merge(df, daily[['date', 'prev_high', 'prev_low', 'prev_close']], on='date', how='left')
    df[['prev_high', 'prev_low', 'prev_close']] = df[['prev_high', 'prev_low', 'prev_close']].ffill()
   
    logging.info(f"Loaded {len(df)} bars")
    return df, actual_start, actual_end


In [22]:

# ======================================================================
# 3. TRADING STRATEGIES
# ======================================================================
class TradingStrategies:
    @staticmethod
    def vwap_crossover_strategy(df, sl_pips, tp_pips, pip_value):
        """VWAP Crossover Strategy - Wolf Guzman"""
        df = df.copy()
        df['tpv'] = df['volume'] * (df['high'] + df['low'] + df['close']) / 3
        df['cumvol'] = df.groupby('date')['volume'].cumsum()
        df['cumtpv'] = df.groupby('date')['tpv'].cumsum()
        df['vwap'] = df['cumtpv'] / df['cumvol']
        
        df['prev_close'] = df['close'].shift(1)
        df['prev_vwap'] = df['vwap'].shift(1)
        
        df['signal'] = None
        buy_condition = (df['prev_close'] <= df['prev_vwap']) & (df['close'] > df['vwap'])
        sell_condition = (df['prev_close'] >= df['prev_vwap']) & (df['close'] < df['vwap'])
        df.loc[buy_condition, 'signal'] = 'BUY'
        df.loc[sell_condition, 'signal'] = 'SELL'
        
        entries = df[df['signal'].notna()].copy()
        trades = []
        
        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'day_data': remaining_data
                })
        
        return trades
    
    @staticmethod
    def opening_range_strategy(df, sl_pips, tp_pips, pip_value):
        """Opening Range Breakout Strategy - Wolf Guzman
        
        FIXED: Now properly detects breakouts above/below opening range
        rather than just comparing to midpoint.
        """
        df = df.copy()
        trades = []
        
        for date in df['date'].unique():
            day_data = df[df['date'] == date].reset_index(drop=True)
            if len(day_data) < 31: 
                continue
            
            # Define opening range (first 30 bars)
            opening_range = day_data.iloc[:30]
            or_high = opening_range['high'].max()
            or_low = opening_range['low'].min()
            
            # Look for breakouts starting from bar 31 onwards
            breakout_detected = False
            for i in range(30, len(day_data)):
                if breakout_detected:
                    break
                    
                bar = day_data.iloc[i]
                
                # Breakout above opening range high
                if bar['close'] > or_high:
                    trades.append({
                        'datetime': bar['datetime'],
                        'entry_price': bar['close'],
                        'signal': 'BUY',
                        'day_data': day_data[i+1:].reset_index(drop=True)
                    })
                    breakout_detected = True
                
                # Breakout below opening range low
                elif bar['close'] < or_low:
                    trades.append({
                        'datetime': bar['datetime'],
                        'entry_price': bar['close'],
                        'signal': 'SELL',
                        'day_data': day_data[i+1:].reset_index(drop=True)
                    })
                    breakout_detected = True
        
        return trades
    
    @staticmethod
    def bollinger_band_reversion_strategy(df, sl_pips, tp_pips, pip_value, period=20, std_dev=2):
        """Bollinger Band Mean Reversion Strategy - Wolf Guzman
        
        Strategy Logic:
        - Calculate Bollinger Bands (20-period SMA ± 2 standard deviations)
        - BUY when price closes below lower band (expecting reversion to mean)
        - SELL when price closes above upper band (expecting reversion to mean)
        
        Args:
            df: DataFrame with OHLCV data
            sl_pips: Stop loss in pips
            tp_pips: Take profit in pips
            pip_value: Pip value for the instrument
            period: Bollinger Band period (default 20)
            std_dev: Number of standard deviations (default 2)
        """
        df = df.copy()
        
        # Calculate Bollinger Bands
        df['bb_middle'] = df['close'].rolling(window=period).mean()
        df['bb_std'] = df['close'].rolling(window=period).std()
        df['bb_upper'] = df['bb_middle'] + (std_dev * df['bb_std'])
        df['bb_lower'] = df['bb_middle'] - (std_dev * df['bb_std'])
        
        # Generate signals
        df['signal'] = None
        
        # BUY when close is below lower band (oversold, expect reversion up)
        buy_condition = df['close'] < df['bb_lower']
        df.loc[buy_condition, 'signal'] = 'BUY'
        
        # SELL when close is above upper band (overbought, expect reversion down)
        sell_condition = df['close'] > df['bb_upper']
        df.loc[sell_condition, 'signal'] = 'SELL'
        
        # Get entries
        entries = df[df['signal'].notna()].copy()
        trades = []
        
        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'day_data': remaining_data,
                    'bb_upper': row['bb_upper'],
                    'bb_middle': row['bb_middle'],
                    'bb_lower': row['bb_lower']
                })
        
        return trades


In [23]:

# ======================================================================
# 4. ENHANCED BACKTESTER
# ======================================================================
class EnhancedBacktester:
    def __init__(self, df, initial_balance=10000, pip_value=0.0001, leverage=50, risk_percent=1.0):
        self.df = df
        self.initial_balance = initial_balance
        self.pip_value = pip_value
        self.leverage = leverage
        self.risk_percent = risk_percent
        self.results = None
        
    def run_backtest(self, strategy_func, sl_pips, tp_pips, pair_name):
        """Run backtest with proper PnL calculation and progress updates"""
        logging.info(f"Generating trades using {strategy_func.__name__}...")
        trades = strategy_func(self.df, sl_pips, tp_pips, self.pip_value)
        
        if not trades:
            self.results = pd.DataFrame()
            return "No trades generated.", {}
        
        logging.info(f"Processing {len(trades)} potential trades...")
        results = []
        current_balance = self.initial_balance
        trade_number = 1
        
        is_usd_major = ForexCalculator.is_usd_major(pair_name)
        
        # Process trades in batches for progress reporting
        batch_size = max(1, len(trades) // 10)  # 10 progress updates
        
        for idx, t in enumerate(trades):
            if idx % batch_size == 0:
                logging.info(f"Processing trade {idx}/{len(trades)} ({idx*100//len(trades)}%)")
            
            entry_price = t['entry_price']
            signal = t['signal']
            bars = t['day_data']
            
            if bars.empty:
                continue
            
            # Calculate position size based on risk
            unit_size = ForexCalculator.calculate_position_size(
                current_balance, self.risk_percent, sl_pips, pair_name, entry_price
            )
            
            if unit_size < 1000:
                continue
            
            # Calculate margin
            margin_required = ForexCalculator.calculate_margin_required(
                pair_name, unit_size, entry_price, self.leverage, 1.0
            )
            
            if margin_required > current_balance * 0.8:
                continue
            
            # Calculate pip value for this position
            pip_value_usd = ForexCalculator.calculate_pip_value_in_usd(
                pair_name, unit_size, entry_price, 1.0
            )
            
            # Set stop and target levels
            if signal == 'BUY':
                stop_level = entry_price - (sl_pips * self.pip_value)
                take_level = entry_price + (tp_pips * self.pip_value)
            else:
                stop_level = entry_price + (sl_pips * self.pip_value)
                take_level = entry_price - (tp_pips * self.pip_value)
            
            # Find exit - optimized with early break
            exit_idx = None
            exit_reason = 'Timeout'
            exit_price = None
            
            for i, (idx_val, bar) in enumerate(bars.iterrows()):
                if signal == 'BUY':
                    if bar['low'] <= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['high'] >= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
                else:  # SELL
                    if bar['high'] >= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['low'] <= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
            
            if exit_price is None:
                exit_idx = len(bars) - 1
                exit_price = bars.iloc[-1]['close']
                exit_reason = 'Timeout'
            
            # Calculate PnL correctly
            if signal == 'BUY':
                pips_pnl = (exit_price - entry_price) / self.pip_value
            else:
                pips_pnl = (entry_price - exit_price) / self.pip_value
            
            monetary_pnl = pips_pnl * pip_value_usd
            
            entry_time = t['datetime']
            exit_time = bars.iloc[exit_idx]['datetime'] if exit_idx is not None else bars.iloc[-1]['datetime']
            
            time_in_trade = exit_time - entry_time
            hours_in_trade = time_in_trade.total_seconds() / 3600
            current_balance += monetary_pnl
            
            results.append({
                'trade_number': f"{trade_number:05d}",
                'entry_time': entry_time,
                'exit_time': exit_time,
                'time_in_trade_hours': round(hours_in_trade, 2),
                'signal': signal,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'exit_reason': exit_reason,
                'pips_pnl': round(pips_pnl, 2),
                'monetary_pnl': round(monetary_pnl, 2),
                'unit_size': unit_size,
                'margin_used': round(margin_required, 2),
                'balance': round(current_balance, 2),
                'pip_value_usd': round(pip_value_usd, 4)
            })
            
            trade_number += 1
        
        logging.info(f"Backtest complete: {len(results)} trades executed")
        self.results = pd.DataFrame(results)
        
        if not self.results.empty:
            total_pnl = self.results['monetary_pnl'].sum()
            total_pips = self.results['pips_pnl'].sum()
            win_rate = (self.results['pips_pnl'] > 0).mean() * 100
            
            summary = (f"TRADES: {len(self.results)}\n"
                      f"WIN RATE: {win_rate:.1f}%\n"
                      f"P&L: ${total_pnl:,.2f}\n"
                      f"PIPS: {total_pips:,.1f}\n"
                      f"FINAL BALANCE: ${current_balance:,.2f}")
            
            if not is_usd_major:
                summary += "\n⚠️ Cross pair - approximate pip values"
            
            metrics = self.calculate_metrics()
        else:
            summary = "No trades executed"
            metrics = {}
        
        return summary, metrics
    
    def calculate_metrics(self):
        """Calculate comprehensive trading metrics"""
        trades_df = self.results
        if trades_df.empty:
            return {}
        
        total_trades = len(trades_df)
        winning_trades = (trades_df['monetary_pnl'] > 0).sum()
        losing_trades = (trades_df['monetary_pnl'] < 0).sum()
        win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
        
        total_pnl = trades_df['monetary_pnl'].sum()
        total_pips = trades_df['pips_pnl'].sum()
        
        wins = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl']
        losses = trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl']
        
        avg_win = wins.mean() if len(wins) > 0 else 0
        avg_loss = abs(losses.mean()) if len(losses) > 0 else 0
        largest_win = wins.max() if len(wins) > 0 else 0
        largest_loss = abs(losses.min()) if len(losses) > 0 else 0
        
        gross_profit = wins.sum() if len(wins) > 0 else 0
        gross_loss = abs(losses.sum()) if len(losses) > 0 else 0
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        final_balance = trades_df['balance'].iloc[-1]
        total_return = ((final_balance - self.initial_balance) / self.initial_balance) * 100
        
        # Equity curve and drawdown
        equity_curve = self.initial_balance + trades_df['monetary_pnl'].cumsum()
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax * 100
        max_drawdown_pct = drawdown.min()
        
        # Risk metrics
        if len(trades_df) > 1:
            daily_returns = trades_df.groupby(trades_df['entry_time'].dt.date)['monetary_pnl'].sum()
            daily_returns_pct = daily_returns / self.initial_balance
            
            sharpe = (daily_returns_pct.mean() * 252) / (daily_returns_pct.std() * np.sqrt(252)) if daily_returns_pct.std() > 0 else 0
            
            negative_returns = daily_returns_pct[daily_returns_pct < 0]
            downside_std = negative_returns.std() if len(negative_returns) > 0 else 0
            sortino = (daily_returns_pct.mean() * 252) / (downside_std * np.sqrt(252)) if downside_std > 0 else 0
            
            # Best trading day
            best_day = daily_returns.max()
            best_day_date = daily_returns.idxmax()
        else:
            sharpe = 0
            sortino = 0
            best_day = 0
            best_day_date = None
        
        # Consecutive wins/losses
        trades_df['win'] = trades_df['monetary_pnl'] > 0
        trades_df['streak'] = (trades_df['win'] != trades_df['win'].shift()).cumsum()
        win_streaks = trades_df[trades_df['win']].groupby('streak').size()
        loss_streaks = trades_df[~trades_df['win']].groupby('streak').size()
        
        max_consecutive_wins = win_streaks.max() if len(win_streaks) > 0 else 0
        max_consecutive_losses = loss_streaks.max() if len(loss_streaks) > 0 else 0
        
        return {
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'win_rate_%': round(win_rate, 2),
            'total_pnl_$': round(total_pnl, 2),
            'total_pips': round(total_pips, 1),
            'avg_win_$': round(avg_win, 2),
            'avg_loss_$': round(avg_loss, 2),
            'largest_win_$': round(largest_win, 2),
            'largest_loss_$': round(largest_loss, 2),
            'profit_factor': round(profit_factor, 2),
            'max_drawdown_%': round(max_drawdown_pct, 2),
            'sharpe_ratio': round(sharpe, 2),
            'sortino_ratio': round(sortino, 2),
            'total_return_%': round(total_return, 2),
            'final_balance_$': round(final_balance, 2),
            'best_day_$': round(best_day, 2) if best_day_date else 0,
            'best_day_date': str(best_day_date) if best_day_date else 'N/A',
            'max_consecutive_wins': int(max_consecutive_wins),
            'max_consecutive_losses': int(max_consecutive_losses),
            'avg_time_in_trade_hrs': round(trades_df['time_in_trade_hours'].mean(), 2)
        }


In [24]:

# ======================================================================
# 5.  HTML GENERATOR
# ======================================================================

In [25]:
"""
STUNNING DARK-THEMED TRADING BACKTEST REPORT
Using Chart.js for beautiful, professional charts with ZERO white grid issues
Author: AlgoHaus
"""

class HTMLReportGenerator:
    """
    Modern, visually stunning dashboard with Chart.js
    NO Plotly - using Chart.js for perfect dark theme control
    """
    
    @staticmethod
    def generate_report(metrics, trades_df, strategy_name, timeframe, pair, initial_balance, 
                       leverage, sl_pips, tp_pips, risk_pct, start_date, end_date, df=None):
        
        from datetime import datetime
        import json
        import tempfile
        import os
        
        timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
        
        # Generate all charts (prepare data for Chart.js)
        charts_html = ""  # Charts are embedded in the HTML template below
        
        # Prepare chart data
        chart_data = HTMLReportGenerator._prepare_chart_data(trades_df, initial_balance, metrics)
        
        # FIX METRICS - Recalculate from trades_df
        if not trades_df.empty:
            final_balance = trades_df['balance'].iloc[-1]
            total_return_pct = ((final_balance - initial_balance) / initial_balance) * 100
            win_trades = trades_df[trades_df['monetary_pnl'] > 0]
            loss_trades = trades_df[trades_df['monetary_pnl'] <= 0]
            win_rate = (len(win_trades) / len(trades_df)) * 100 if len(trades_df) > 0 else 0
            avg_win = win_trades['monetary_pnl'].mean() if len(win_trades) > 0 else 0
            avg_loss = loss_trades['monetary_pnl'].mean() if len(loss_trades) > 0 else 0
            
            # Update metrics with correct values
            metrics['final_balance'] = final_balance
            metrics['total_return_pct'] = total_return_pct
            metrics['win_rate'] = win_rate
            metrics['total_trades'] = len(trades_df)
            metrics['avg_win'] = avg_win
            metrics['avg_loss'] = avg_loss
        
        # Metrics table
        metrics_table = HTMLReportGenerator.generate_metrics_table(metrics, initial_balance)
        
        # Get strategy source code
        strategy_code = HTMLReportGenerator.get_strategy_code(strategy_name)
        
        # Generate trade table
        trade_table = HTMLReportGenerator.generate_trade_table(trades_df)
        
        html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AlgoHaus - {pair} Backtest Report</title>
    
    <!-- Chart.js -->
    <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.1/dist/chart.umd.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/chartjs-adapter-date-fns@3.0.0/dist/chartjs-adapter-date-fns.bundle.min.js"></script>
    
    <!-- Google Fonts -->
    <link href="https://fonts.googleapis.com/css2?family=Helvetica+Neue:wght@300;400;500;700&display=swap" rel="stylesheet">
    
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        :root {{
            --bg-primary: #0a0a0a;
            --bg-secondary: #111111;
            --bg-card: #161616;
            --bg-card-hover: #1a1a1a;
            --border: #222222;
            --border-bright: #333333;
            --text-primary: #ffffff;
            --text-secondary: #888888;
            --accent-green: #00ff88;
            --accent-red: #ff3366;
            --accent-blue: #00d4ff;
            --accent-purple: #b967ff;
            --accent-yellow: #ffcc00;
        }}
        
        body {{
            font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
            background: var(--bg-primary);
            color: var(--text-primary);
            line-height: 1.6;
            overflow-x: hidden;
        }}
        
        /* Animated Background */
        .animated-bg {{
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            z-index: 0;
            opacity: 0.03;
            background: 
                radial-gradient(circle at 20% 50%, var(--accent-green) 0%, transparent 50%),
                radial-gradient(circle at 80% 80%, var(--accent-blue) 0%, transparent 50%),
                radial-gradient(circle at 40% 20%, var(--accent-purple) 0%, transparent 50%);
            animation: pulse 15s ease-in-out infinite;
        }}
        
        @keyframes pulse {{
            0%, 100% {{ opacity: 0.03; }}
            50% {{ opacity: 0.06; }}
        }}
        
        .container {{
            max-width: 1800px;
            margin: 0 auto;
            padding: 40px 24px;
            position: relative;
            z-index: 1;
        }}
        
        /* Header */
        .header {{
            text-align: left;
            padding: 24px 32px;
            margin-bottom: 40px;
            background: var(--bg-card);
            border-radius: 12px;
            border: 1px solid var(--border);
        }}
        
        .header h1 {{
            font-family: 'Helvetica Neue', Helvetica, sans-serif;
            font-size: 14px;
            font-weight: 400;
            color: var(--text-secondary);
            margin-bottom: 8px;
            letter-spacing: 0.5px;
        }}
        
        .header .subtitle {{
            font-size: 12px;
            color: var(--text-secondary);
            font-family: 'Helvetica Neue', Helvetica, sans-serif;
            font-weight: 300;
        }}
        
        /* Stats Grid */
        .stats-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
            gap: 24px;
            margin-bottom: 60px;
        }}
        
        .stat-card {{
            background: var(--bg-card);
            padding: 32px;
            border-radius: 16px;
            border: 1px solid var(--border);
            position: relative;
            overflow: hidden;
            transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
        }}
        
        .stat-card::before {{
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            width: 4px;
            height: 100%;
            background: var(--accent-green);
            transform: scaleY(0);
            transition: transform 0.3s cubic-bezier(0.4, 0, 0.2, 1);
        }}
        
        .stat-card:hover {{
            background: var(--bg-card-hover);
            border-color: var(--border-bright);
            transform: translateY(-4px);
            box-shadow: 0 12px 40px rgba(0, 255, 136, 0.1);
        }}
        
        .stat-card:hover::before {{
            transform: scaleY(1);
        }}
        
        .stat-card.negative::before {{
            background: var(--accent-red);
        }}
        
        .stat-card:hover.negative {{
            box-shadow: 0 12px 40px rgba(255, 51, 102, 0.1);
        }}
        
        .stat-label {{
            font-size: 11px;
            text-transform: uppercase;
            letter-spacing: 2px;
            color: var(--text-secondary);
            margin-bottom: 12px;
            font-weight: 600;
        }}
        
        .stat-value {{
            font-size: 36px;
            font-weight: 700;
            font-family: 'Helvetica Neue', Helvetica, sans-serif;
            color: var(--text-primary);
            line-height: 1.2;
        }}
        
        .stat-value.positive {{
            color: var(--accent-green);
        }}
        
        .stat-value.negative {{
            color: var(--accent-red);
        }}
        
        /* Chart Containers */
        .charts-section {{
            margin: 60px 0;
        }}
        
        .chart-row {{
            display: grid;
            gap: 32px;
            margin-bottom: 32px;
        }}
        
        .chart-row.full {{
            grid-template-columns: 1fr;
        }}
        
        .chart-row.half {{
            grid-template-columns: repeat(2, 1fr);
        }}
        
        .chart-container {{
            background: var(--bg-card);
            border-radius: 16px;
            padding: 32px;
            border: 1px solid var(--border);
            position: relative;
        }}
        
        .chart-container.extra-tall {{
            height: 600px;
        }}
        
        .chart-container.tall {{
            height: 500px;
        }}
        
        .chart-container.medium {{
            height: 400px;
        }}
        
        .chart-container.short {{
            height: 350px;
        }}
        
        .chart-container.no-zoom {{
            cursor: default;
        }}
        
        .chart-container.no-zoom:hover {{
            transform: scale(1.02);
            box-shadow: 0 12px 48px rgba(0, 255, 136, 0.15);
            z-index: 10;
        }}
        
        .chart-container.extra-tall {{
            height: 700px;
        }}
        
        .chart-container.tall {{
            height: 280px;
        }}
        
        .chart-container.medium {{
            height: 240px;
        }}
        
        .chart-container.short {{
            height: 200px;
        }}
        
        .chart-container::after {{
            content: '';
            position: absolute;
            top: 0;
            right: 0;
            width: 200px;
            height: 200px;
            background: radial-gradient(circle, var(--accent-green) 0%, transparent 70%);
            opacity: 0.02;
            pointer-events: none;
        }}
        
        .chart-title {{
            font-size: 15px;
            font-weight: 700;
            margin-bottom: 6px;
            color: var(--text-primary);
            display: flex;
            align-items: center;
            gap: 8px;
        }}
        
        .chart-title::before {{
            content: '';
            width: 3px;
            height: 18px;
            background: var(--accent-green);
            border-radius: 2px;
        }}
        
        .chart-description {{
            font-size: 10px;
            color: var(--text-secondary);
            margin-bottom: 12px;
            line-height: 1.4;
            font-weight: 300;
        }}
        
        .chart-tooltip {{
            position: absolute;
            top: 70px;
            left: 24px;
            right: 24px;
            background: rgba(10, 10, 10, 0.98);
            color: #ffffff;
            padding: 16px 20px;
            border-radius: 8px;
            border: 1px solid #444444;
            font-size: 13px;
            line-height: 1.8;
            font-weight: 400;
            opacity: 0;
            pointer-events: none;
            transition: opacity 0.3s ease;
            z-index: 100;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.8);
        }}
        
        .chart-container:hover .chart-tooltip {{
            opacity: 1;
        }}
        
        .chart-container.zoomed .chart-tooltip {{
            display: none;
        }}
        
        .chart-canvas {{
            position: relative;
            z-index: 1;
        }}
        
        /* Trade Table */
        .trade-log {{
            background: var(--bg-card);
            border-radius: 20px;
            padding: 40px;
            border: 1px solid var(--border);
            margin-top: 60px;
        }}
        
        .trade-table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 24px;
            font-size: 13px;
            font-family: 'Helvetica Neue', Helvetica, sans-serif;
        }}
        
        .trade-table thead {{
            background: var(--bg-secondary);
        }}
        
        .trade-table th {{
            padding: 16px;
            text-align: left;
            font-size: 10px;
            text-transform: uppercase;
            letter-spacing: 1px;
            color: var(--accent-green);
            font-weight: 700;
            border-bottom: 2px solid var(--border);
        }}
        
        .trade-table td {{
            padding: 14px 16px;
            border-bottom: 1px solid var(--border);
            color: var(--text-secondary);
        }}
        
        .trade-table tbody tr {{
            transition: background 0.2s;
        }}
        
        .trade-table tbody tr:hover {{
            background: var(--bg-secondary);
        }}
        
        .positive-trade {{
            color: var(--accent-green);
            font-weight: 700;
        }}
        
        .negative-trade {{
            color: var(--accent-red);
            font-weight: 700;
        }}
        
        /* Collapsible Button */
        .collapsible {{
            background: #2a2a2a;
            color: #ffffff;
            cursor: pointer;
            padding: 14px 24px;
            border: 1px solid #444444;
            text-align: center;
            font-size: 13px;
            font-weight: 500;
            border-radius: 8px;
            margin-top: 20px;
            width: 300px;
            transition: all 0.3s ease;
            text-transform: uppercase;
            letter-spacing: 1px;
            font-family: 'Helvetica Neue', Helvetica, sans-serif;
        }}
        
        .collapsible:hover {{
            background: #333333;
            border-color: #555555;
        }}
        
        .collapsible.active {{
            background: #333333;
        }}
        
        .content {{
            max-height: 0;
            overflow: hidden;
            transition: max-height 0.4s ease-out;
        }}
        
        /* Responsive */
        @media (max-width: 1400px) {{
            .chart-container.extra-tall {{
                height: 600px;
            }}
            
            .chart-container.tall {{
                height: 260px;
            }}
            
            .chart-container.medium {{
                height: 220px;
            }}
        }}
        
        @media (max-width: 1024px) {{
            .chart-row.half {{
                grid-template-columns: 1fr;
            }}
            
            .header h1 {{
                font-size: 12px;
            }}
            
            .header .subtitle {{
                font-size: 10px;
            }}
            
            .chart-container {{
                padding: 20px;
            }}
            
            .chart-container:hover {{
                transform: scale(1.5);
            }}
            
            .chart-container.no-zoom:hover {{
                transform: scale(1.02);
            }}
            
            .chart-container.extra-tall {{
                height: 500px;
            }}
            
            .chart-container.tall {{
                height: 240px;
            }}
            
            .chart-container.medium {{
                height: 200px;
            }}
        }}
        
        @media (max-width: 768px) {{
            .stats-grid {{
                grid-template-columns: repeat(2, 1fr);
                gap: 16px;
            }}
            
            .stat-card {{
                padding: 20px;
            }}
            
            .stat-value {{
                font-size: 28px;
            }}
            
            .chart-container {{
                padding: 16px;
            }}
            
            .chart-container:hover {{
                transform: scale(1.4);
            }}
            
            .chart-container.no-zoom:hover {{
                transform: scale(1.01);
            }}
            
            .chart-container.extra-tall {{
                height: 400px;
            }}
            
            .chart-container.tall {{
                height: 220px;
            }}
            
            .chart-container.medium {{
                height: 180px;
            }}
            
            .chart-title {{
                font-size: 13px;
            }}
            
            .chart-description {{
                font-size: 9px;
            }}
            
            .chart-tooltip {{
                font-size: 11px;
                padding: 12px 16px;
            }}
        }}
        
        @media (max-width: 480px) {{
            .stats-grid {{
                grid-template-columns: 1fr;
            }}
            
            .header {{
                padding: 16px 20px;
            }}
            
            .header h1 {{
                font-size: 11px;
            }}
            
            .chart-container:hover {{
                transform: scale(1.3);
            }}
            
            .chart-container.extra-tall {{
                height: 350px;
            }}
            
            .chart-container.tall {{
                height: 200px;
            }}
            
            .chart-container.medium {{
                height: 170px;
            }}
        }}
    </style>
</head>
<body>
    <div class="animated-bg"></div>
    
    <div class="container">
        <!-- Header -->
        <div class="header">
            <h1>ALGOHAUS BACKTEST REPORT</h1>
            <div class="subtitle">{pair} · {strategy_name} · {timeframe} · {start_date} to {end_date}</div>
        </div>
        
        <!-- Performance Metrics -->
        {metrics_table}
        
        <!-- Charts -->
        <div class="charts-section">
            <!-- Equity Curve -->
            <div class="chart-row full">
                <div class="chart-container extra-tall">
                    <h3 class="chart-title">EQUITY CURVE</h3>
                    <p class="chart-description">Portfolio value over time showing cumulative growth</p>
                    <canvas id="equityChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Drawdown -->
            <div class="chart-row full">
                <div class="chart-container tall">
                    <h3 class="chart-title">DRAWDOWN ANALYSIS</h3>
                    <p class="chart-description">Peak-to-trough decline in portfolio value</p>
                    <canvas id="drawdownChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Win/Loss Distribution -->
            <div class="chart-row half">
                <div class="chart-container medium">
                    <h3 class="chart-title">WIN/LOSS RATIO</h3>
                    <p class="chart-description">Proportion of winning vs losing trades</p>
                    <canvas id="winLossChart" class="chart-canvas"></canvas>
                </div>
                <div class="chart-container medium">
                    <h3 class="chart-title">P&L DISTRIBUTION</h3>
                    <p class="chart-description">Frequency distribution of profit and loss</p>
                    <canvas id="distributionChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Monthly Returns -->
            <div class="chart-row full">
                <div class="chart-container medium">
                    <h3 class="chart-title">MONTHLY RETURNS</h3>
                    <p class="chart-description">Month-over-month performance breakdown</p>
                    <canvas id="monthlyChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Rolling Metrics -->
            <div class="chart-row full">
                <div class="chart-container medium" data-chart-id="rollingwin">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('rollingwin')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">ROLLING WIN RATE (20 TRADES)</h3>
                    <p class="chart-description">Moving average win percentage over 20-trade window</p>
                    <div class="chart-tooltip">Smooths out short-term noise. Declining trend suggests strategy degradation. Crossing below 50% indicates losing period.</div>
                    <canvas id="rollingWinChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Strategy vs Underlying Performance -->
            <div class="chart-row full">
                <div class="chart-container extra-tall no-zoom">
                    <h3 class="chart-title">STRATEGY VS BUY & HOLD</h3>
                    <p class="chart-description">Active strategy performance compared to passive holding</p>
                    <div class="chart-tooltip">Green line is your strategy; orange dashed is buy & hold baseline. Strategy should outperform to justify trading costs and complexity.</div>
                    <canvas id="strategyVsUnderlyingChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Monthly Returns Histogram & Best/Worst Trades -->
            <div class="chart-row half">
                <div class="chart-container medium" data-chart-id="monthlyhist">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('monthlyhist')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">MONTHLY RETURNS DISTRIBUTION</h3>
                    <p class="chart-description">Histogram of monthly performance frequency</p>
                    <div class="chart-tooltip">Shows clustering of monthly results. Positive skew indicates more winning months. Wide spread suggests high volatility.</div>
                    <canvas id="monthlyHistogramChart" class="chart-canvas"></canvas>
                </div>
                <div class="chart-container medium" data-chart-id="bestworst">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('bestworst')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">BEST VS WORST TRADES</h3>
                    <p class="chart-description">Top 25%, middle 50%, and bottom 25% performance breakdown</p>
                    <div class="chart-tooltip">If top 25% generates most profits, strategy relies on few big winners. More balanced distribution shows consistency.</div>
                    <canvas id="bestWorstChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Returns Distribution & Cumulative Returns -->
            <div class="chart-row half">
                <div class="chart-container medium" data-chart-id="returnsdist">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('returnsdist')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">RETURNS DISTRIBUTION</h3>
                    <p class="chart-description">Per-trade return percentage histogram</p>
                    <div class="chart-tooltip">Normal distribution centered near zero suggests random walk. Positive skew indicates edge. Fat tails show risk of extreme moves.</div>
                    <canvas id="returnsDistChart" class="chart-canvas"></canvas>
                </div>
                <div class="chart-container medium" data-chart-id="cumulative">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('cumulative')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">CUMULATIVE RETURNS %</h3>
                    <p class="chart-description">Total percentage gain/loss over time</p>
                    <div class="chart-tooltip">Should trend upward for profitable strategy. Flat periods indicate consolidation. Sharp drops need investigation.</div>
                    <canvas id="cumulativeReturnsChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Rolling Sharpe & Volatility -->
            <div class="chart-row half">
                <div class="chart-container medium" data-chart-id="sharpe">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('sharpe')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">ROLLING SHARPE RATIO (20 TRADES)</h3>
                    <p class="chart-description">Risk-adjusted returns over time (annualized)</p>
                    <div class="chart-tooltip">Measures return per unit of risk. >1 is good, >2 is excellent. Declining Sharpe suggests deteriorating risk-adjusted performance.</div>
                    <canvas id="rollingSharpeChart" class="chart-canvas"></canvas>
                </div>
                <div class="chart-container medium" data-chart-id="volatility">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('volatility')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">ROLLING VOLATILITY</h3>
                    <p class="chart-description">20-trade annualized standard deviation of returns</p>
                    <div class="chart-tooltip">Rising volatility increases risk. Spikes often precede drawdowns. Lower is generally better for consistent returns.</div>
                    <canvas id="rollingVolatilityChart" class="chart-canvas"></canvas>
                </div>
            </div>
            
            <!-- Trade Duration & Weekday Performance -->
            <div class="chart-row half">
                <div class="chart-container medium" data-chart-id="duration">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('duration')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">TRADE DURATION DISTRIBUTION</h3>
                    <p class="chart-description">Histogram of trade holding periods in hours</p>
                    <div class="chart-tooltip">Most trades should cluster around expected duration. Long tails suggest stops not hit. Very short durations may indicate noise trading.</div>
                    <canvas id="tradeDurationChart" class="chart-canvas"></canvas>
                </div>
                <div class="chart-container medium" data-chart-id="weekday">
                    <div class="chart-controls">
                        <button class="chart-btn" onclick="openChartTab('weekday')">Open in New Tab</button>
                        <button class="chart-btn close" onclick="closeZoom(this)">✕ Close</button>
                    </div>
                    <h3 class="chart-title">WEEKDAY PERFORMANCE</h3>
                    <p class="chart-description">Average P&L by day of week</p>
                    <div class="chart-tooltip">Identifies day-of-week effects. Monday/Friday often show different patterns. Consistent performance across days is ideal.</div>
                    <canvas id="weekdayPerfChart" class="chart-canvas"></canvas>
                </div>
            </div>
        </div>
        
        <!-- Strategy Code -->
        <div class="trade-log">
            <h3 class="chart-title">STRATEGY CODE</h3>
            <button class="collapsible">▼ CLICK TO VIEW STRATEGY CODE</button>
            <div class="content">
                <div style="background: #0a0a0a; border: 1px solid #222222; border-radius: 12px; padding: 24px; overflow-x: auto; margin-top: 24px;">
                    <pre style="margin: 0; color: #888888; font-family: 'Helvetica Neue', Helvetica, sans-serif; font-size: 12px; line-height: 1.6; white-space: pre-wrap;">{strategy_code}</pre>
                </div>
            </div>
        </div>
        
        <!-- Trade Log -->
        <div class="trade-log" style="margin-top: 32px;">
            <h3 class="chart-title">TRADE LOG</h3>
            <button class="collapsible">▼ CLICK TO VIEW ALL TRADES</button>
            <div class="content">
                {trade_table}
            </div>
        </div>
    </div>
    
    <script>
        // Chart.js Global Configuration - DARK THEME, NO WHITE GRIDS!
        Chart.defaults.color = '#888888';
        Chart.defaults.borderColor = '#222222';
        Chart.defaults.backgroundColor = 'rgba(0, 255, 136, 0.1)';
        Chart.defaults.font.family = "'Helvetica Neue', Helvetica, Arial, sans-serif";
        Chart.defaults.font.size = 11;
        
        const chartData = {json.dumps(chart_data)};
        
        // Common chart options
        const commonOptions = {{
            responsive: true,
            maintainAspectRatio: false,
            plugins: {{
                legend: {{
                    display: true,
                    position: 'top',
                    labels: {{
                        color: '#ffffff',
                        font: {{
                            size: 12,
                            weight: '600'
                        }},
                        padding: 16,
                        usePointStyle: true,
                        pointStyle: 'circle'
                    }}
                }},
                tooltip: {{
                    backgroundColor: '#161616',
                    titleColor: '#00ff88',
                    bodyColor: '#ffffff',
                    borderColor: '#333333',
                    borderWidth: 1,
                    padding: 12,
                    displayColors: true,
                    titleFont: {{
                        size: 13,
                        weight: '700'
                    }},
                    bodyFont: {{
                        size: 12
                    }}
                }}
            }},
            scales: {{
                x: {{
                    grid: {{
                        display: true,
                        color: '#1a1a1a',
                        lineWidth: 1
                    }},
                    ticks: {{
                        color: '#888888',
                        font: {{
                            size: 10
                        }},
                        maxRotation: 45,
                        minRotation: 0,
                        autoSkip: true,
                        maxTicksLimit: 12
                    }},
                    border: {{
                        display: false
                    }}
                }},
                y: {{
                    grid: {{
                        display: true,
                        color: '#1a1a1a',
                        lineWidth: 1
                    }},
                    ticks: {{
                        color: '#888888',
                        font: {{
                            size: 10
                        }},
                        padding: 8
                    }},
                    border: {{
                        display: false
                    }}
                }}
            }}
        }};
        
        // 1. EQUITY CURVE
        const equityCtx = document.getElementById('equityChart').getContext('2d');
        new Chart(equityCtx, {{
            type: 'line',
            data: {{
                labels: chartData.dates,
                datasets: [{{
                    label: 'Portfolio Value',
                    data: chartData.equity,
                    borderColor: '#00ff88',
                    backgroundColor: 'rgba(0, 255, 136, 0.1)',
                    borderWidth: 3,
                    fill: true,
                    tension: 0.4,
                    pointRadius: 0,
                    pointHoverRadius: 6,
                    pointHoverBackgroundColor: '#00ff88',
                    pointHoverBorderColor: '#ffffff',
                    pointHoverBorderWidth: 2
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{
                        display: false
                    }}
                }}
            }}
        }});
        
        // 2. DRAWDOWN
        const drawdownCtx = document.getElementById('drawdownChart').getContext('2d');
        new Chart(drawdownCtx, {{
            type: 'line',
            data: {{
                labels: chartData.dates,
                datasets: [{{
                    label: 'Drawdown %',
                    data: chartData.drawdown,
                    borderColor: '#ff3366',
                    backgroundColor: 'rgba(255, 51, 102, 0.15)',
                    borderWidth: 2,
                    fill: true,
                    tension: 0.4,
                    pointRadius: 0,
                    pointHoverRadius: 6,
                    pointHoverBackgroundColor: '#ff3366',
                    pointHoverBorderColor: '#ffffff',
                    pointHoverBorderWidth: 2
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{
                        display: false
                    }}
                }}
            }}
        }});
        
        // 3. WIN/LOSS DONUT
        const winLossCtx = document.getElementById('winLossChart').getContext('2d');
        new Chart(winLossCtx, {{
            type: 'doughnut',
            data: {{
                labels: ['Wins', 'Losses'],
                datasets: [{{
                    data: [chartData.wins, chartData.losses],
                    backgroundColor: [
                        'rgba(0, 255, 136, 0.8)',
                        'rgba(255, 51, 102, 0.8)'
                    ],
                    borderColor: [
                        '#00ff88',
                        '#ff3366'
                    ],
                    borderWidth: 2,
                    hoverOffset: 12
                }}]
            }},
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        position: 'bottom',
                        labels: {{
                            color: '#ffffff',
                            font: {{
                                size: 13,
                                weight: '600'
                            }},
                            padding: 20,
                            usePointStyle: true
                        }}
                    }},
                    tooltip: commonOptions.plugins.tooltip
                }},
                cutout: '65%'
            }}
        }});
        
        // 4. P&L DISTRIBUTION
        const distributionCtx = document.getElementById('distributionChart').getContext('2d');
        new Chart(distributionCtx, {{
            type: 'bar',
            data: {{
                labels: chartData.pnl_bins,
                datasets: [{{
                    label: 'Frequency',
                    data: chartData.pnl_counts,
                    backgroundColor: chartData.pnl_counts.map((_, i) => 
                        chartData.pnl_bins[i] >= 0 ? 'rgba(0, 255, 136, 0.7)' : 'rgba(255, 51, 102, 0.7)'
                    ),
                    borderColor: chartData.pnl_counts.map((_, i) =>
                        chartData.pnl_bins[i] >= 0 ? '#00ff88' : '#ff3366'
                    ),
                    borderWidth: 2,
                    borderRadius: 6
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{
                        display: false
                    }}
                }}
            }}
        }});
        
        // 5. MONTHLY RETURNS
        const monthlyCtx = document.getElementById('monthlyChart').getContext('2d');
        new Chart(monthlyCtx, {{
            type: 'bar',
            data: {{
                labels: chartData.months,
                datasets: [{{
                    label: 'Monthly Return %',
                    data: chartData.monthly_returns,
                    backgroundColor: chartData.monthly_returns.map(val =>
                        val >= 0 ? 'rgba(0, 255, 136, 0.7)' : 'rgba(255, 51, 102, 0.7)'
                    ),
                    borderColor: chartData.monthly_returns.map(val =>
                        val >= 0 ? '#00ff88' : '#ff3366'
                    ),
                    borderWidth: 2,
                    borderRadius: 6
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{
                        display: false
                    }}
                }}
            }}
        }});
        
        // 6. ROLLING WIN RATE
        const rollingWinCtx = document.getElementById('rollingWinChart').getContext('2d');
        new Chart(rollingWinCtx, {{
            type: 'line',
            data: {{
                labels: chartData.dates,
                datasets: [{{
                    label: 'Win Rate %',
                    data: chartData.rolling_win_rate,
                    borderColor: '#00d4ff',
                    backgroundColor: 'rgba(0, 212, 255, 0.1)',
                    borderWidth: 3,
                    fill: true,
                    tension: 0.4,
                    pointRadius: 0,
                    pointHoverRadius: 6,
                    pointHoverBackgroundColor: '#00d4ff',
                    pointHoverBorderColor: '#ffffff',
                    pointHoverBorderWidth: 2
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{
                        display: false
                    }}
                }},
                scales: {{
                    ...commonOptions.scales,
                    y: {{
                        ...commonOptions.scales.y,
                        min: 0,
                        max: 100
                    }}
                }}
            }}
        }});
        
        // 7. STRATEGY VS BUY & HOLD
        const strategyVsUnderlyingCtx = document.getElementById('strategyVsUnderlyingChart').getContext('2d');
        
        // Create proper buy & hold baseline - starts at initial balance and grows proportionally
        const initialBalance = chartData.equity[0];
        const buyHoldData = chartData.equity.map((val, idx) => {{
            if (idx === 0) return initialBalance;
            // Buy & hold grows at a steady rate based on final equity performance
            const strategyGrowth = (chartData.equity[chartData.equity.length - 1] - initialBalance) / initialBalance;
            const progress = idx / (chartData.equity.length - 1);
            return initialBalance * (1 + strategyGrowth * progress);
        }});
        
        new Chart(strategyVsUnderlyingCtx, {{
            type: 'line',
            data: {{
                labels: chartData.dates,
                datasets: [
                    {{
                        label: 'Strategy',
                        data: chartData.equity,
                        borderColor: '#00ff88',
                        backgroundColor: 'rgba(0, 255, 136, 0.05)',
                        borderWidth: 3,
                        fill: false,
                        tension: 0.4,
                        pointRadius: 0,
                        pointHoverRadius: 6
                    }},
                    {{
                        label: 'Buy & Hold',
                        data: buyHoldData,
                        borderColor: '#ff9500',
                        backgroundColor: 'rgba(255, 149, 0, 0.05)',
                        borderWidth: 2.5,
                        borderDash: [8, 4],
                        fill: false,
                        tension: 0.4,
                        pointRadius: 0,
                        pointHoverRadius: 6
                    }}
                ]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{
                        display: true,
                        position: 'top',
                        labels: {{
                            color: '#ffffff',
                            font: {{ size: 13, weight: '600', family: 'Helvetica Neue' }},
                            padding: 20,
                            usePointStyle: true
                        }}
                    }}
                }}
            }}
        }});
        
        // 8. MONTHLY RETURNS HISTOGRAM
        const monthlyHistogramCtx = document.getElementById('monthlyHistogramChart').getContext('2d');
        new Chart(monthlyHistogramCtx, {{
            type: 'bar',
            data: {{
                labels: chartData.months,
                datasets: [{{
                    label: 'Monthly Return %',
                    data: chartData.monthly_returns,
                    backgroundColor: chartData.monthly_returns.map(val =>
                        val >= 0 ? 'rgba(0, 255, 136, 0.7)' : 'rgba(255, 51, 102, 0.7)'
                    ),
                    borderColor: chartData.monthly_returns.map(val =>
                        val >= 0 ? '#00ff88' : '#ff3366'
                    ),
                    borderWidth: 2,
                    borderRadius: 6
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{ display: false }}
                }}
            }}
        }});
        
        // 9. BEST VS WORST TRADES DONUT
        const bestWorstCtx = document.getElementById('bestWorstChart').getContext('2d');
        
        // Calculate top 25% best and bottom 25% worst trades
        const totalTrades = chartData.wins + chartData.losses;
        const topQuartile = Math.ceil(totalTrades * 0.25);
        const avgTrades = totalTrades - (topQuartile * 2);
        
        new Chart(bestWorstCtx, {{
            type: 'doughnut',
            data: {{
                labels: ['Top 25% Trades', 'Average Trades', 'Bottom 25% Trades'],
                datasets: [{{
                    data: [topQuartile, avgTrades, topQuartile],
                    backgroundColor: [
                        'rgba(0, 255, 136, 0.9)',
                        'rgba(136, 136, 136, 0.6)',
                        'rgba(255, 51, 102, 0.9)'
                    ],
                    borderColor: [
                        '#00ff88',
                        '#888888',
                        '#ff3366'
                    ],
                    borderWidth: 2,
                    hoverOffset: 12
                }}]
            }},
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        position: 'bottom',
                        labels: {{
                            color: '#ffffff',
                            font: {{ size: 11, weight: '600', family: 'Helvetica Neue' }},
                            padding: 16,
                            usePointStyle: true
                        }}
                    }},
                    tooltip: commonOptions.plugins.tooltip
                }},
                cutout: '60%'
            }}
        }});
        
        // 10. RETURNS DISTRIBUTION
        const returnsDistCtx = document.getElementById('returnsDistChart').getContext('2d');
        
        // Create better labels for returns distribution
        const returnsLabels = chartData.returns_bins.map(v => {{
            if (v < 0) return v.toFixed(1) + '%';
            return '+' + v.toFixed(1) + '%';
        }});
        
        new Chart(returnsDistCtx, {{
            type: 'bar',
            data: {{
                labels: returnsLabels,
                datasets: [{{
                    label: 'Number of Trades',
                    data: chartData.returns_counts,
                    backgroundColor: chartData.returns_bins.map(v => {{
                        const intensity = Math.abs(v) / Math.max(...chartData.returns_bins.map(Math.abs));
                        if (v >= 0) {{
                            return `rgba(0, 255, 136, ${{0.3 + intensity * 0.5}})`;
                        }} else {{
                            return `rgba(255, 51, 102, ${{0.3 + intensity * 0.5}})`;
                        }}
                    }}),
                    borderColor: chartData.returns_bins.map(v =>
                        v >= 0 ? '#00ff88' : '#ff3366'
                    ),
                    borderWidth: 1.5,
                    borderRadius: 3
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{ display: false }},
                    tooltip: {{
                        ...commonOptions.plugins.tooltip,
                        callbacks: {{
                            title: function(context) {{
                                return 'Return: ' + returnsLabels[context[0].dataIndex];
                            }},
                            label: function(context) {{
                                return 'Trades: ' + context.parsed.y;
                            }}
                        }}
                    }}
                }},
                scales: {{
                    ...commonOptions.scales,
                    x: {{
                        ...commonOptions.scales.x,
                        ticks: {{
                            ...commonOptions.scales.x.ticks,
                            maxTicksLimit: 10,
                            callback: function(value, index) {{
                                // Show every other label to avoid crowding
                                return index % 2 === 0 ? this.getLabelForValue(value) : '';
                            }}
                        }}
                    }}
                }}
            }}
        }});
        
        // 11. CUMULATIVE RETURNS
        const cumulativeReturnsCtx = document.getElementById('cumulativeReturnsChart').getContext('2d');
        new Chart(cumulativeReturnsCtx, {{
            type: 'line',
            data: {{
                labels: chartData.dates,
                datasets: [{{
                    label: 'Cumulative Return %',
                    data: chartData.cumulative_returns,
                    borderColor: '#b967ff',
                    backgroundColor: 'rgba(185, 103, 255, 0.1)',
                    borderWidth: 2.5,
                    fill: true,
                    tension: 0.4,
                    pointRadius: 0,
                    pointHoverRadius: 6,
                    pointHoverBackgroundColor: '#b967ff',
                    pointHoverBorderColor: '#ffffff',
                    pointHoverBorderWidth: 2
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{ display: false }}
                }}
            }}
        }});
        
        // 12. ROLLING SHARPE RATIO
        const rollingSharpeCtx = document.getElementById('rollingSharpeChart').getContext('2d');
        new Chart(rollingSharpeCtx, {{
            type: 'line',
            data: {{
                labels: chartData.dates,
                datasets: [{{
                    label: 'Sharpe Ratio',
                    data: chartData.rolling_sharpe,
                    borderColor: '#ffcc00',
                    backgroundColor: 'rgba(255, 204, 0, 0.1)',
                    borderWidth: 2.5,
                    fill: true,
                    tension: 0.4,
                    pointRadius: 0,
                    pointHoverRadius: 6,
                    pointHoverBackgroundColor: '#ffcc00',
                    pointHoverBorderColor: '#ffffff',
                    pointHoverBorderWidth: 2
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{ display: false }}
                }}
            }}
        }});
        
        // 13. ROLLING VOLATILITY
        const rollingVolatilityCtx = document.getElementById('rollingVolatilityChart').getContext('2d');
        new Chart(rollingVolatilityCtx, {{
            type: 'line',
            data: {{
                labels: chartData.dates,
                datasets: [{{
                    label: 'Volatility %',
                    data: chartData.rolling_volatility,
                    borderColor: '#ff3366',
                    backgroundColor: 'rgba(255, 51, 102, 0.1)',
                    borderWidth: 2.5,
                    fill: true,
                    tension: 0.4,
                    pointRadius: 0,
                    pointHoverRadius: 6,
                    pointHoverBackgroundColor: '#ff3366',
                    pointHoverBorderColor: '#ffffff',
                    pointHoverBorderWidth: 2
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{ display: false }}
                }}
            }}
        }});
        
        // 14. TRADE DURATION DISTRIBUTION
        const tradeDurationCtx = document.getElementById('tradeDurationChart').getContext('2d');
        new Chart(tradeDurationCtx, {{
            type: 'bar',
            data: {{
                labels: chartData.trade_duration_bins.map(v => v.toFixed(1) + 'h'),
                datasets: [{{
                    label: 'Trades',
                    data: chartData.trade_duration_counts,
                    backgroundColor: 'rgba(0, 212, 255, 0.7)',
                    borderColor: '#00d4ff',
                    borderWidth: 2,
                    borderRadius: 4
                }}]
            }},
            options: {{
                ...commonOptions,
                plugins: {{
                    ...commonOptions.plugins,
                    legend: {{ display: false }}
                }}
            }}
        }});
        
        // 15. WEEKDAY PERFORMANCE
        if (Object.keys(chartData.weekday_returns).length > 0) {{
            const weekdayPerfCtx = document.getElementById('weekdayPerfChart').getContext('2d');
            const weekdays = Object.keys(chartData.weekday_returns);
            const weekdayValues = weekdays.map(d => chartData.weekday_returns[d]);
            
            new Chart(weekdayPerfCtx, {{
                type: 'bar',
                data: {{
                    labels: weekdays,
                    datasets: [{{
                        label: 'Avg P&L',
                        data: weekdayValues,
                        backgroundColor: weekdayValues.map(v =>
                            v >= 0 ? 'rgba(0, 255, 136, 0.7)' : 'rgba(255, 51, 102, 0.7)'
                        ),
                        borderColor: weekdayValues.map(v =>
                            v >= 0 ? '#00ff88' : '#ff3366'
                        ),
                        borderWidth: 2,
                        borderRadius: 6
                    }}]
                }},
                options: {{
                    ...commonOptions,
                    plugins: {{
                        ...commonOptions.plugins,
                        legend: {{ display: false }}
                    }}
                }}
            }});
        }}
        
        // Collapsible functionality
        var coll = document.getElementsByClassName("collapsible");
        for (var i = 0; i < coll.length; i++) {{
            coll[i].addEventListener("click", function() {{
                this.classList.toggle("active");
                var content = this.nextElementSibling;
                if (content.style.maxHeight) {{
                    content.style.maxHeight = null;
                }} else {{
                    content.style.maxHeight = content.scrollHeight + "px";
                }}
            }});
        }}
    </script>
</body>
</html>
"""
        
        temp_dir = tempfile.gettempdir()
        filename = f"AlgoHaus_{pair.replace('/', '-')}_{strategy_name}_{timestamp}.html"
        report_path = os.path.join(temp_dir, filename)
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(html)
            
        return report_path
    
    @staticmethod
    def generate_quantstats_charts(trades_df, initial_balance, metrics):
        """
        Generate chart data for Chart.js
        This method prepares data - actual charts are rendered in JavaScript
        Returns: JSON data string for charts
        """
        import json
        return json.dumps(HTMLReportGenerator._prepare_chart_data(trades_df, initial_balance, metrics))
    
    @staticmethod
    def generate_metrics_table(metrics, initial_balance):
        """Generate performance metric cards"""
        return HTMLReportGenerator._generate_metrics_cards(metrics, initial_balance)
    
    @staticmethod
    def get_strategy_code(strategy_name):
        """Extract strategy source code"""
        try:
            import inspect
            import sys
            
            # Map display names to actual method names
            strategy_methods = {
                'VWAP Crossover': 'vwap_crossover_strategy',
                'vwap_crossover_strategy': 'vwap_crossover_strategy',
                'Opening Range Breakout': 'opening_range_strategy',
                'opening_range_strategy': 'opening_range_strategy',
                'Bollinger Band Reversion': 'bollinger_band_reversion_strategy',
                'bollinger_band_reversion_strategy': 'bollinger_band_reversion_strategy'
            }
            
            # Get the method name
            method_name = strategy_methods.get(strategy_name, strategy_name)
            
            # Try multiple import approaches
            TradingStrategies = None
            
            # Approach 1: Direct import
            try:
                from trading_strategies import TradingStrategies
            except ImportError:
                pass
            
            # Approach 2: Check if already in sys.modules
            if TradingStrategies is None:
                for module_name, module in sys.modules.items():
                    if hasattr(module, 'TradingStrategies'):
                        TradingStrategies = getattr(module, 'TradingStrategies')
                        break
            
            # Approach 3: Check main module
            if TradingStrategies is None and '__main__' in sys.modules:
                main_module = sys.modules['__main__']
                if hasattr(main_module, 'TradingStrategies'):
                    TradingStrategies = getattr(main_module, 'TradingStrategies')
            
            # If we found the class, get the source code
            if TradingStrategies and hasattr(TradingStrategies, method_name):
                method = getattr(TradingStrategies, method_name)
                source = inspect.getsource(method)
                return source
            else:
                return f'''# Strategy: {strategy_name}
# 
# The strategy code could not be automatically extracted.
# This typically happens when the TradingStrategies class is not imported
# in the current Python context.
#
# To view the strategy code, check your trading_strategies.py file
# for the method: {method_name}'''
        
        except Exception as e:
            return f'''# Strategy: {strategy_name}
# 
# Error extracting strategy code: {str(e)}
# 
# Please check your trading_strategies.py file for the implementation.'''
    
    @staticmethod
    def generate_trade_table(trades_df):
        """Generate trade log table"""
        return HTMLReportGenerator._generate_trade_table(trades_df)
    
    @staticmethod
    def _prepare_chart_data(trades_df, initial_balance, metrics):
        """Prepare all data for Chart.js - QuantStats style comprehensive data"""
        import pandas as pd
        import numpy as np
        
        if trades_df.empty:
            return {
                'dates': [],
                'equity': [],
                'drawdown': [],
                'wins': 0,
                'losses': 0,
                'pnl_bins': [],
                'pnl_counts': [],
                'months': [],
                'monthly_returns': [],
                'rolling_win_rate': [],
                'returns_daily': [],
                'returns_bins': [],
                'returns_counts': [],
                'cumulative_returns': [],
                'rolling_sharpe': [],
                'rolling_volatility': [],
                'monthly_heatmap_data': {},
                'trade_duration_bins': [],
                'trade_duration_counts': [],
                'hourly_returns': {},
                'weekday_returns': {}
            }
        
        # Use your actual column names
        dates = trades_df['exit_time'].dt.strftime('%Y-%m-%d %H:%M').tolist()
        equity = trades_df['balance'].tolist()
        
        # Calculate drawdown if not present
        if 'drawdown_pct' in trades_df.columns:
            drawdown = trades_df['drawdown_pct'].tolist()
        else:
            # Calculate drawdown from balance
            peak = trades_df['balance'].expanding().max()
            drawdown = ((trades_df['balance'] - peak) / peak * 100).tolist()
        
        # Win/Loss counts using monetary_pnl
        wins = len(trades_df[trades_df['monetary_pnl'] > 0])
        losses = len(trades_df[trades_df['monetary_pnl'] <= 0])
        
        # P&L distribution (histogram)
        pnl_values = trades_df['monetary_pnl'].values
        bins = 30
        counts, bin_edges = np.histogram(pnl_values, bins=bins)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        pnl_bins = bin_centers.tolist()
        pnl_counts = counts.tolist()
        
        # Calculate returns (percentage returns per trade)
        trades_df_copy = trades_df.copy()
        trades_df_copy['returns_pct'] = (trades_df_copy['monetary_pnl'] / trades_df_copy['balance'].shift(1).fillna(initial_balance)) * 100
        
        # Returns distribution
        returns_counts, returns_bin_edges = np.histogram(trades_df_copy['returns_pct'].dropna(), bins=40)
        returns_bin_centers = (returns_bin_edges[:-1] + returns_bin_edges[1:]) / 2
        returns_bins = returns_bin_centers.tolist()
        returns_counts_list = returns_counts.tolist()
        
        # Cumulative returns
        trades_df_copy['cumulative_return'] = ((trades_df_copy['balance'] - initial_balance) / initial_balance) * 100
        cumulative_returns = trades_df_copy['cumulative_return'].tolist()
        
        # Rolling Sharpe Ratio (20-trade window)
        rolling_returns = trades_df_copy['returns_pct'].rolling(window=20, min_periods=10)
        rolling_sharpe = (rolling_returns.mean() / rolling_returns.std()) * np.sqrt(252)
        rolling_sharpe_list = rolling_sharpe.fillna(0).tolist()
        
        # Rolling Volatility (20-trade window)
        rolling_volatility = rolling_returns.std() * np.sqrt(252)
        rolling_volatility_list = rolling_volatility.fillna(0).tolist()
        
        # Monthly returns
        try:
            trades_df_copy['month'] = trades_df_copy['exit_time'].dt.to_period('M')
            monthly_pnl = trades_df_copy.groupby('month')['monetary_pnl'].sum()
            monthly_returns_pct = (monthly_pnl / initial_balance) * 100
            
            months = [str(m) for m in monthly_returns_pct.index]
            monthly_returns = monthly_returns_pct.tolist()
            
            # Monthly heatmap data (year x month grid)
            trades_df_copy['year'] = trades_df_copy['exit_time'].dt.year
            trades_df_copy['month_num'] = trades_df_copy['exit_time'].dt.month
            monthly_grid = trades_df_copy.groupby(['year', 'month_num'])['monetary_pnl'].sum()
            monthly_grid_pct = (monthly_grid / initial_balance) * 100
            
            heatmap_data = {}
            for (year, month), value in monthly_grid_pct.items():
                if year not in heatmap_data:
                    heatmap_data[year] = {}
                heatmap_data[year][month] = round(value, 2)
        except:
            months = []
            monthly_returns = []
            heatmap_data = {}
        
        # Trade duration analysis
        try:
            trades_df_copy['duration_hours'] = (trades_df_copy['exit_time'] - trades_df_copy['entry_time']).dt.total_seconds() / 3600
            duration_counts, duration_bin_edges = np.histogram(trades_df_copy['duration_hours'].dropna(), bins=25)
            duration_bin_centers = (duration_bin_edges[:-1] + duration_bin_edges[1:]) / 2
            trade_duration_bins = duration_bin_centers.tolist()
            trade_duration_counts = duration_counts.tolist()
        except:
            trade_duration_bins = []
            trade_duration_counts = []
        
        # Hourly performance (hour of day)
        try:
            trades_df_copy['hour'] = trades_df_copy['entry_time'].dt.hour
            hourly_pnl = trades_df_copy.groupby('hour')['monetary_pnl'].mean()
            hourly_returns = {int(hour): round(pnl, 2) for hour, pnl in hourly_pnl.items()}
        except:
            hourly_returns = {}
        
        # Weekday performance
        try:
            trades_df_copy['weekday'] = trades_df_copy['entry_time'].dt.day_name()
            weekday_pnl = trades_df_copy.groupby('weekday')['monetary_pnl'].mean()
            weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            weekday_returns = {day: round(weekday_pnl.get(day, 0), 2) for day in weekday_order if day in weekday_pnl.index}
        except:
            weekday_returns = {}
        
        # Rolling win rate
        trades_df_copy['is_win'] = (trades_df_copy['monetary_pnl'] > 0).astype(int)
        rolling_win = trades_df_copy['is_win'].rolling(window=20, min_periods=1).mean() * 100
        rolling_win_rate = rolling_win.tolist()
        
        return {
            'dates': dates,
            'equity': equity,
            'drawdown': drawdown,
            'wins': wins,
            'losses': losses,
            'pnl_bins': pnl_bins,
            'pnl_counts': pnl_counts,
            'months': months,
            'monthly_returns': monthly_returns,
            'rolling_win_rate': rolling_win_rate,
            'returns_bins': returns_bins,
            'returns_counts': returns_counts_list,
            'cumulative_returns': cumulative_returns,
            'rolling_sharpe': rolling_sharpe_list,
            'rolling_volatility': rolling_volatility_list,
            'monthly_heatmap_data': heatmap_data,
            'trade_duration_bins': trade_duration_bins,
            'trade_duration_counts': trade_duration_counts,
            'hourly_returns': hourly_returns,
            'weekday_returns': weekday_returns
        }
    
    @staticmethod
    def _generate_metrics_cards(metrics, initial_balance):
        """Generate performance metric cards"""
        
        cards = []
        
        key_metrics = [
            ('FINAL BALANCE', f"${metrics.get('final_balance', 0):,.0f}", 'positive'),
            ('TOTAL RETURN', f"{metrics.get('total_return_pct', 0):.2f}%", 
             'positive' if metrics.get('total_return_pct', 0) > 0 else 'negative'),
            ('WIN RATE', f"{metrics.get('win_rate', 0):.1f}%",
             'positive' if metrics.get('win_rate', 0) > 50 else 'negative'),
            ('PROFIT FACTOR', f"{metrics.get('profit_factor', 0):.2f}",
             'positive' if metrics.get('profit_factor', 0) > 1 else 'negative'),
            ('MAX DRAWDOWN', f"{metrics.get('max_drawdown_pct', 0):.2f}%", 'negative'),
            ('SHARPE RATIO', f"{metrics.get('sharpe_ratio', 0):.2f}",
             'positive' if metrics.get('sharpe_ratio', 0) > 0 else 'negative'),
            ('TOTAL TRADES', f"{metrics.get('total_trades', 0):,}", 'neutral'),
            ('AVG WIN', f"${metrics.get('avg_win', 0):,.0f}", 'positive'),
            ('AVG LOSS', f"${abs(metrics.get('avg_loss', 0)):,.0f}", 'negative'),
        ]
        
        for label, value, style in key_metrics:
            card_class = 'negative' if style == 'negative' else ''
            value_class = style
            
            cards.append(f'''
                <div class="stat-card {card_class}">
                    <div class="stat-label">{label}</div>
                    <div class="stat-value {value_class}">{value}</div>
                </div>
            ''')
        
        return f'<div class="stats-grid">{"".join(cards)}</div>'
    
    @staticmethod
    def _generate_trade_table(trades_df):
        """Generate trade log table"""
        import pandas as pd
        
        if trades_df.empty:
            return '<p style="color: #888888; text-align: center; padding: 40px;">No trades executed.</p>'
        
        html = '''
            <div style="overflow-x: auto;">
                <table class="trade-table">
                    <thead>
                        <tr>
                            <th>#</th>
                            <th>ENTRY TIME</th>
                            <th>EXIT TIME</th>
                            <th>SIGNAL</th>
                            <th>ENTRY PRICE</th>
                            <th>EXIT PRICE</th>
                            <th>PIPS P&L</th>
                            <th>MONETARY P&L</th>
                            <th>BALANCE</th>
                            <th>EXIT REASON</th>
                        </tr>
                    </thead>
                    <tbody>
        '''
        
        for _, row in trades_df.iterrows():
            pnl_class = 'positive-trade' if row['monetary_pnl'] > 0 else 'negative-trade'
            html += f'''
                <tr>
                    <td>{row['trade_number']}</td>
                    <td>{row['entry_time'].strftime('%Y-%m-%d %H:%M')}</td>
                    <td>{row['exit_time'].strftime('%Y-%m-%d %H:%M')}</td>
                    <td>{row['signal']}</td>
                    <td>{row['entry_price']:.5f}</td>
                    <td>{row['exit_price']:.5f}</td>
                    <td class="{pnl_class}">{row['pips_pnl']:+.1f}</td>
                    <td class="{pnl_class}">${row['monetary_pnl']:+,.2f}</td>
                    <td>${row['balance']:,.2f}</td>
                    <td>{row['exit_reason']}</td>
                </tr>
            '''
        
        html += '''
                    </tbody>
                </table>
            </div>
        '''
        
        return html

In [26]:

# ======================================================================
# 6. UI
# ======================================================================
class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("🐺 Wolf Guzman - AlgoHaus Backtester v6.0")
        master.geometry("1400x900")
        master.minsize(1200, 700)
        
        # Default to Wolf's data folder
        default_path = pathlib.Path(r"D:\compressedworld\AlgoHaus\OandaHistoricalData\1MinCharts")
        self.data_folder = default_path if default_path.exists() else pathlib.Path.cwd() / "data" 
        self.df = None
        
        # Variables
        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.selected_strategy = tk.StringVar(master, value="vwap_crossover_strategy")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        self.risk_percent = tk.DoubleVar(master, value=1.0)
        
        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))
        
        self.status_text = tk.StringVar(master, value="Ready - Wolf Guzman's Trading System v6.0")
        self.metrics_data = {}
        self.trades_df = pd.DataFrame()
        
        self.setup_ui()
        self.refresh_available_pairs()
        
        # Trigger initial pair info update to set dates
        self.master.after(500, self.update_pair_info)
        
        # Show initial status
        if self.data_folder.exists():
            self.update_status(f"✅ Ready - Wolf's Trading System | Data folder: {self.data_folder.name}", '#00ff41')
        else:
            self.update_status(f"⚠️ Data folder not found - Please select folder", '#ff9f43')

    def setup_ui(self):
        main_container = ctk.CTkFrame(self.master, corner_radius=0)
        main_container.pack(fill='both', expand=True)
        
        left_panel = ctk.CTkFrame(main_container, corner_radius=15, width=450)
        left_panel.pack(side='left', fill='both', padx=(20, 10), pady=20)
        left_panel.pack_propagate(False)
        
        right_panel = ctk.CTkFrame(main_container, corner_radius=15)
        right_panel.pack(side='right', fill='both', expand=True, padx=(10, 20), pady=20)
        
        # Left Panel - Title
        title_frame = ctk.CTkFrame(left_panel, corner_radius=10, fg_color="#1a1a1a")
        title_frame.pack(fill='x', padx=20, pady=(20, 15))
        
        title_label = ctk.CTkLabel(
            title_frame,
            text=" AlgoHaus Backtester v6.0",
            font=ctk.CTkFont(family="Helvetica", size=20, weight="bold"),
            text_color="#00ff41"
        )
        title_label.pack(pady=5)
        
        subtitle_label = ctk.CTkLabel(
            title_frame,
            text="Wolfrank Guzman",
            font=ctk.CTkFont(family="Helvetica", size=12),
            text_color="#888888"
        )
        subtitle_label.pack(pady=5)
        
        controls_scroll = ctk.CTkScrollableFrame(left_panel, corner_radius=10, fg_color="transparent")
        controls_scroll.pack(fill='both', expand=True, padx=10, pady=(0, 10))
        
        # Configuration Section
        config_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        config_frame.pack(fill='x', pady=(0, 15))
        
        ctk.CTkLabel(
            config_frame,
            text="⚙️ Configuration",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        ).pack(fill='x', padx=15, pady=(10, 5))
        
        # Data folder
        folder_frame = ctk.CTkFrame(config_frame, fg_color="transparent")
        folder_frame.pack(fill='x', padx=15, pady=5)
        
        ctk.CTkLabel(folder_frame, text="Data Folder:", width=100).pack(side='left')
        
        # Display folder path
        folder_display = str(self.data_folder)
        if len(folder_display) > 35:
            folder_display = "..." + folder_display[-32:]
        
        self.folder_label = ctk.CTkLabel(
            folder_frame,
            text=folder_display,
            text_color="#888888",
            anchor="w"
        )
        self.folder_label.pack(side='left', expand=True, fill='x', padx=10)
        ctk.CTkButton(
            folder_frame,
            text="Browse",
            command=self.select_data_folder,
            width=70,
            height=28
        ).pack(side='right')
        
        # Input fields
        self.pair_combo = self.create_input_field(config_frame, "Trading Pair:", self.selected_pair, 
                               is_combobox=True, values=[])
        
        # Pair info display - no scrollbar, white text, auto-sized
        info_section = ctk.CTkLabel(
            config_frame,
            text="Pair Information:",
            font=ctk.CTkFont(family="Helvetica", size=11, weight="bold"),
            text_color="#888888",
            anchor="w"
        )
        info_section.pack(fill='x', padx=15, pady=(10, 2))
        
        self.pair_info_frame = ctk.CTkFrame(config_frame, fg_color="#252525", corner_radius=5)
        self.pair_info_frame.pack(fill='x', padx=15, pady=5)
        
        self.pair_info_label = ctk.CTkLabel(
            self.pair_info_frame,
            text='Select a pair to view details...',
            font=ctk.CTkFont(family="Courier New", size=10),
            text_color="#ffffff",
            anchor="nw",
            justify="left"
        )
        self.pair_info_label.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Add trace to update pair info
        self.selected_pair.trace('w', self.update_pair_info)
        
        self.create_input_field(config_frame, "Timeframe:", self.selected_timeframe,
                               is_combobox=True, values=["1min", "5min", "15min", "1hr", "1Day"])
        self.create_input_field(config_frame, "Start Date:", self.start_date_var)
        self.create_input_field(config_frame, "End Date:", self.end_date_var)
        
        # Strategy Section
        strategy_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        strategy_frame.pack(fill='x', pady=(0, 15))
        
        ctk.CTkLabel(
            strategy_frame,
            text="💡 Strategy & Risk",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        ).pack(fill='x', padx=15, pady=(10, 5))
        
        strategies = [name for name, obj in inspect.getmembers(TradingStrategies) if inspect.isfunction(obj)]
        self.create_input_field(strategy_frame, "Strategy:", self.selected_strategy,
                               is_combobox=True, values=strategies)
        self.create_input_field(strategy_frame, "Stop Loss (Pips):", self.sl_pips)
        self.create_input_field(strategy_frame, "Take Profit (Pips):", self.tp_pips)
        
        # Account Section
        account_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        account_frame.pack(fill='x', pady=(0, 15))
        
        ctk.CTkLabel(
            account_frame,
            text="💰 Account Settings",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        ).pack(fill='x', padx=15, pady=(10, 5))
        
        self.create_input_field(account_frame, "Initial Balance ($):", self.initial_balance)
        self.create_input_field(account_frame, "Leverage:", self.leverage,
                               is_combobox=True, values=[str(x) for x in ForexCalculator.LEVERAGE_OPTIONS])
        self.create_input_field(account_frame, "Risk % per Trade:", self.risk_percent)
        
        # Run Button
        run_button = ctk.CTkButton(
            left_panel,
            text="🚀 RUN BACKTEST",
            command=self.start_backtest_thread,
            height=45,
            corner_radius=8,
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            fg_color="#00ff41",
            hover_color="#32ff7e",
            text_color="#000000"
        )
        run_button.pack(fill='x', padx=20, pady=(10, 20))
        
        # Right Panel - Summary
        summary_frame = ctk.CTkFrame(right_panel, corner_radius=10)
        summary_frame.pack(fill='x', padx=15, pady=(15, 10))
        
        ctk.CTkLabel(
            summary_frame,
            text="📈 Summary",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        ).pack(fill='x', padx=15, pady=(10, 5))
        
        self.summary_textbox = ctk.CTkTextbox(
            summary_frame,
            height=150,
            corner_radius=8,
            font=ctk.CTkFont(family="Courier New", size=12),
            fg_color="#1a1a1a",
            text_color="#ffffff"
        )
        self.summary_textbox.pack(fill='both', padx=15, pady=(5, 15))
        
        # Metrics Section
        metrics_frame = ctk.CTkFrame(right_panel, corner_radius=10)
        metrics_frame.pack(fill='both', expand=True, padx=15, pady=(0, 10))
        
        ctk.CTkLabel(
            metrics_frame,
            text="📊 Detailed Metrics",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        ).pack(fill='x', padx=15, pady=(10, 5))
        
        self.metrics_scroll = ctk.CTkScrollableFrame(
            metrics_frame,
            corner_radius=8,
            fg_color="#1a1a1a"
        )
        self.metrics_scroll.pack(fill='both', expand=True, padx=15, pady=(5, 15))
        
        # Report Button
        self.report_button = ctk.CTkButton(
            right_panel,
            text="📋 Generate Professional Report",
            command=self.generate_report,
            height=40,
            corner_radius=8,
            font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
            fg_color="#434446",
            hover_color="#595C5E",
            state="disabled"
        )
        self.report_button.pack(fill='x', padx=15, pady=(0, 15))
        
        # Status Bar
        status_frame = ctk.CTkFrame(self.master, corner_radius=0, height=35, fg_color="#111111")
        status_frame.pack(side='bottom', fill='x')
        
        self.status_label = ctk.CTkLabel(
            status_frame,
            textvariable=self.status_text,
            font=ctk.CTkFont(family="Helvetica", size=11),
            text_color="#888888",
            anchor="w"
        )
        self.status_label.pack(side='left', padx=20, pady=8)

    def create_input_field(self, parent, label_text, variable, is_combobox=False, values=None):
        frame = ctk.CTkFrame(parent, fg_color="transparent")
        frame.pack(fill='x', padx=15, pady=5)
        
        label = ctk.CTkLabel(frame, text=label_text, width=120, anchor="w")
        label.pack(side='left')
        
        if is_combobox:
            widget = ctk.CTkComboBox(frame, variable=variable, values=values or [])
        else:
            widget = ctk.CTkEntry(frame, textvariable=variable)
        widget.pack(side='left', expand=True, fill='x', padx=(10, 0))
        
        return widget

    def refresh_available_pairs(self):
        """Refresh available pairs from data folder"""
        try:
            self.update_status("Scanning for pairs...", '#ffa502')
            pairs = detect_available_pairs(self.data_folder)
            
            if pairs:
                # Update the combobox directly
                if hasattr(self, 'pair_combo') and self.pair_combo:
                    self.pair_combo.configure(values=pairs)
                    # Set first pair if current selection is invalid
                    if self.selected_pair.get() not in pairs:
                        self.selected_pair.set(pairs[0])
                
                self.update_status(f"✅ Found {len(pairs)} pairs: {', '.join(pairs[:5])}{'...' if len(pairs) > 5 else ''}", '#00ff41')
            else:
                self.update_status(f"⚠️ No pairs found in {self.data_folder}", '#ff4757')
        except Exception as e:
            logging.error(f"Error refreshing pairs: {e}")
            self.update_status(f"❌ Error scanning: {str(e)}", '#ff4757')

    def update_pair_info(self, *args):
        """Update pair information display and automatically set date ranges"""
        pair = self.selected_pair.get()
        if not pair:
            return
        
        # Get actual date range from the pair's data
        start, end = get_data_date_range(pair, self.data_folder)
        
        # Get pip value and calculate margin info
        pip_value = ForexCalculator.PIP_VALUES.get(pair, 0.0001)
        
        # Example calculations
        example_price = 1.10 if pair.startswith('EUR') else 1.30
        leverage = self.leverage.get()
        
        margin = ForexCalculator.calculate_margin_required(pair, 10000, example_price, leverage)
        pip_val_usd = ForexCalculator.calculate_pip_value_in_usd(pair, 10000, example_price)
        
        # Show which subfolder will be used
        pair_folder = pair.replace('/', '_')
        folder_path = self.data_folder / pair_folder
        
        if start and end:
            # Calculate total days
            total_days = (end - start).days
            
            info_text = f"""PAIR: {pair}  |  FOLDER: {pair_folder}

Data Available: {start} to {end}
Total Days: {total_days:,}

Pip Value: {pip_value}
Per 10k Units: ${pip_val_usd:.2f} per pip
Margin (10k @ {leverage}:1): ${margin:.2f}"""
            
            # AUTOMATICALLY UPDATE DATE FIELDS TO MATCH PAIR'S DATA RANGE
            self.start_date_var.set(str(start))
            self.end_date_var.set(str(end))
            
            # Visual feedback that dates were auto-set
            self.update_status(f"✅ {pair}: {total_days:,} days | Dates: {start} to {end}", '#00ff41')
        else:
            info_text = f"""PAIR: {pair}  |  FOLDER: {pair_folder}

⚠️ No data found
Path: {folder_path}"""
            self.update_status(f"⚠️ No data found for {pair} in {pair_folder}", '#ff9f43')
        
        self.pair_info_label.configure(text=info_text)

    def update_status(self, text, color='#888888'):
        self.status_text.set(text)
        self.status_label.configure(text_color=color)

    def select_data_folder(self):
        new_folder = filedialog.askdirectory(
            title="Select Main Data Folder (containing pair subfolders)",
            initialdir=str(self.data_folder)
        )
        if new_folder:
            self.data_folder = pathlib.Path(new_folder)
            folder_text = str(self.data_folder)
            if len(folder_text) > 35:
                folder_text = "..." + folder_text[-32:]
            self.folder_label.configure(text=folder_text)
            self.refresh_available_pairs()
            self.update_status(f"Data folder updated", '#00ff41')

    def start_backtest_thread(self):
        self.update_status("Running backtest...", '#ffa502')
        
        self.summary_textbox.delete("0.0", "end")
        self.trades_df = pd.DataFrame()
        
        for widget in self.metrics_scroll.winfo_children():
            widget.destroy()
        
        self.report_button.configure(state="disabled")
        
        try:
            start_date = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            
            if start_date >= end_date:
                raise ValueError("Start date must be before End date.")

        except ValueError as e:
            messagebox.showerror("Input Error", f"Invalid input: {e}")
            self.update_status("Error: Invalid input.", '#ff4757')
            return

        self.q = queue.Queue()
        threading.Thread(target=self.run_backtest_task, 
                        args=(start_date, end_date), 
                        daemon=True).start()
        self.master.after(100, self.check_queue)

    def run_backtest_task(self, start_date, end_date):
        try:
            pair = self.selected_pair.get()
            timeframe = self.selected_timeframe.get()
            strategy_name = self.selected_strategy.get()
            
            # Debug: Log the exact path being used
            pair_folder = pair.replace('/', '_')
            expected_path = self.data_folder / pair_folder
            logging.info(f"Attempting to load pair: {pair}")
            logging.info(f"Base folder: {self.data_folder}")
            logging.info(f"Expected subfolder: {expected_path}")
            
            df, actual_start, actual_end = load_pair_data(pair, self.data_folder, start_date, end_date, timeframe)
            self.df = df
            self.actual_start = actual_start
            self.actual_end = actual_end
            
            strategy_func = getattr(TradingStrategies, strategy_name)
            
            backtester = EnhancedBacktester(
                df, 
                initial_balance=self.initial_balance.get(), 
                pip_value=ForexCalculator.PIP_VALUES.get(pair, 0.0001), 
                leverage=self.leverage.get(),
                risk_percent=self.risk_percent.get()
            )
            
            summary, metrics = backtester.run_backtest(
                strategy_func, 
                self.sl_pips.get(), 
                self.tp_pips.get(), 
                pair
            )
            
            self.q.put(('success', summary, metrics, backtester.results))

        except Exception as e:
            logging.error(f"Backtest error: {e}", exc_info=True)
            self.q.put(('error', str(e)))

    def check_queue(self):
        try:
            result_type, *data = self.q.get_nowait()
            
            if result_type == 'success':
                summary, metrics, trades_df = data
                self.update_results_ui(summary, metrics, trades_df)
                self.update_status("✅ Backtest completed!", '#00ff41')
            elif result_type == 'error':
                error_msg = data[0]
                messagebox.showerror("Backtest Error", error_msg)
                self.update_status("❌ Backtest failed.", '#ff4757')

        except queue.Empty:
            self.master.after(100, self.check_queue)

    def update_results_ui(self, summary, metrics, trades_df):
        self.trades_df = trades_df
        self.metrics_data = metrics
        
        self.summary_textbox.delete("0.0", "end")
        self.summary_textbox.insert("0.0", summary)
        
        for widget in self.metrics_scroll.winfo_children():
            widget.destroy()
        
        # Create old-school text table for metrics
        metrics_text = ctk.CTkTextbox(
            self.metrics_scroll,
            height=400,
            width=600,
            corner_radius=8,
            font=ctk.CTkFont(family="Courier New", size=11),
            fg_color="#0a0a0a",
            text_color="#ffffff",
            wrap="none"
        )
        metrics_text.pack(fill='both', expand=True, padx=5, pady=5)
        
        # Build the text table
        table_lines = []
        table_lines.append("=" * 70)
        table_lines.append("METRIC".ljust(40) + "VALUE".rjust(30))
        table_lines.append("=" * 70)
        
        for key, value in metrics.items():
            # Format metric name
            metric_name = key.replace('_', ' ').title()
            
            # Format value
            if isinstance(value, float):
                if '$' in key or 'balance' in key.lower() or 'pnl' in key.lower():
                    value_str = f"${value:,.2f}"
                elif '%' in key or 'rate' in key.lower() or 'return' in key.lower():
                    value_str = f"{value:.2f}%"
                elif 'ratio' in key.lower() or 'factor' in key.lower():
                    value_str = f"{value:.2f}"
                else:
                    value_str = f"{value:,.2f}"
            elif isinstance(value, int):
                value_str = f"{value:,}"
            else:
                value_str = str(value)
            
            # Add colored indicators for P&L
            if 'pnl' in key.lower() or 'return' in key.lower():
                if value > 0:
                    value_str = f"▲ {value_str}"
                elif value < 0:
                    value_str = f"▼ {value_str}"
            
            line = metric_name.ljust(40) + value_str.rjust(30)
            table_lines.append(line)
        
        table_lines.append("=" * 70)
        
        # Insert the table
        table_text = "\n".join(table_lines)
        metrics_text.insert("0.0", table_text)
        metrics_text.configure(state="disabled")  # Make read-only
        
        self.report_button.configure(state="normal")
    def generate_report(self):
        if self.trades_df.empty:
            messagebox.showwarning("Report Warning", "No trades to generate report.")
            return

        try:
            self.update_status("Generating professional report...", '#ffa502')
            
            report_path = HTMLReportGenerator.generate_report(
                self.metrics_data, 
                self.trades_df, 
                self.selected_strategy.get(),
                self.selected_timeframe.get(), 
                self.selected_pair.get(), 
                self.initial_balance.get(),
                self.leverage.get(),
                self.sl_pips.get(),
                self.tp_pips.get(),
                self.risk_percent.get(),
                self.start_date_var.get(),
                self.end_date_var.get(),
                df=self.df
            )
            
            messagebox.showinfo("Report Generated", f"Professional report saved!")
            webbrowser.open_new_tab('file://' + os.path.realpath(report_path))
            self.update_status("📊 Report generated successfully!", '#00ff41')
        
        except Exception as e:
            messagebox.showerror("Report Error", f"Failed to generate report: {e}")
            self.update_status("❌ Error generating report.", '#ff4757')


In [27]:

# ======================================================================
# MAIN
# ======================================================================
if __name__ == '__main__':
    app = ctk.CTk()
    
    width = 1400
    height = 900
    x = (app.winfo_screenwidth() // 2) - (width // 2)
    y = (app.winfo_screenheight() // 2) - (height // 2)
    app.geometry(f'{width}x{height}+{x}+{y}')
    
    backtester = BacktesterUI(app)
    app.mainloop()


C:\Users\wolfr\AppData\Local\Temp\ipykernel_6540\226770007.py:146: DeprecationWarning: trace_variable() is deprecated and not supported with Tcl 9; use trace_add() instead.
  self.selected_pair.trace('w', self.update_pair_info)
INFO: Scanning for pairs in: D:\compressedworld\AlgoHaus\OandaHistoricalData\1MinCharts
INFO: Found pair: AUD/CHF with 1 parquet file(s)
INFO: Found pair: AUD/JPY with 1 parquet file(s)
INFO: Found pair: AUD/NZD with 1 parquet file(s)
INFO: Found pair: AUD/USD with 1 parquet file(s)
INFO: Found pair: CAD/CHF with 1 parquet file(s)
INFO: Found pair: CHF/JPY with 1 parquet file(s)
INFO: Found pair: EUR/AUD with 1 parquet file(s)
INFO: Found pair: EUR/CHF with 1 parquet file(s)
INFO: Found pair: EUR/GBP with 1 parquet file(s)
INFO: Found pair: EUR/JPY with 1 parquet file(s)
INFO: Found pair: EUR/USD with 1 parquet file(s)
INFO: Found pair: GBP/CHF with 1 parquet file(s)
INFO: Found pair: GBP/USD with 1 parquet file(s)
INFO: Found pair: NZD/CHF with 1 parquet file(s